In [1]:
import sys
sys.path.insert(0, 'C:/Users/Alex Koh/Dropbox/Python/FIFA/fut-master')
#sys.path.insert(0, 'C:/Users/Alex/Dropbox/Python/FIFA/fut-master')

import fut
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime, timedelta
import random
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import lxml.html as lh
from IPython.display import display
import sqlite3
import statistics
#from apscheduler.schedulers.background import BackgroundScheduler

#don't truncate
pd.set_option('display.max_colwidth', -1, 'display.max_columns', 999, 'display.max_rows', 999)
pd.set_option('float_format', '{:,.2f}'.format)
pd.options.mode.chained_assignment = None  # default='warn'

## database of players
players_db = fut.core.players()

## connect to sqlite3
conn = sqlite3.connect('C:/Users/Alex Koh/Dropbox/Python/FIFA/player_db.db')
#conn = sqlite3.connect('C:/Users/Alex/Dropbox/Python/FIFA/player_db.db')
cursor = conn.cursor()

## create scheduler
#scheduler = BackgroundScheduler()

## proxies (maybe move to csv file)
'''proxies = {'http': random.choice(['http://35.182.54.53:3128',
                'http://36.68.32.102:3128',
                'http://181.129.50.162:36796',
                'http://51.77.229.110:8080',
                'http://51.77.229.171:8080',
                'http://122.102.28.187:49828',
                'http://201.16.212.79:80',
                'http://177.53.57.154:46416',
                'http://90.182.160.242:57637',
                'http://80.48.119.28:8080',
                'http://35.182.193.6:3128',
                'http://176.196.254.206:41986',
                'http://206.189.163.84:80',
                'http://157.230.149.189:80',
                'http://157.230.212.164:8080',
                'http://206.189.234.211:80',
                #'http://157.230.216.214:80',
                'http://157.230.140.12:8080'])}'''
proxies = {
    #'http': 'http://122.102.28.187:49828'
    'http': 'http://68.183.97.187:3128'
}

In [66]:
# Make the HTTP request through the session.

r = requests.get('http://www.showmemyip.com/', proxies=proxies)
print(proxies)
# Check if the proxy was indeed used (the text should contain the proxy IP).
print(r.text)

{'http': 'http://68.183.97.187:3128'}
<!DOCTYPE html>
<html lang="en">
	<head>
		<meta charset="utf-8" />
		<meta name="viewport" content="width=device-width, initial-scale=1.0">
		<title>Show Me My IP! - Your IP address is: 68.183.97.187</title>

		<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />

		<meta name="category" content="internet" />
		<meta name="keywords" content="IP Address, router, IP, reverse DNS lookup, hostname, host, name, network address translation, NAT, what is my IP, router IP, external IP, show me my IP" />
		<meta name="description" content="Find your computer or router's IP address easily." />

		<script>
			(function (i, s, o, g, r, a, m) {
				i['GoogleAnalyticsObject'] = r; i[r] = i[r] || function () {
					(i[r].q = i[r].q || []).push(arguments)
				}, i[r].l = 1 * new Date(); a = s.createElement(o),
				m = s.getElementsByTagName(o)[0]; a.async = 1; a.src = g; m.parentNode.insertBefore(a, m)
			})(window, document, 'script', '//www.goo

In [47]:
def futbin_to_db(min_rating=75, max_rating=99, min_price=200, max_price=15000000):
    
    time_start = datetime.now()
    print("Start Time: " + str(time_start))
        
        ## 
    futbin_id_df = futbin_market_scan(min_rating=min_rating, max_rating=max_rating, min_price=min_price, max_price=max_price)
    
        ##
    write_futbin_hist_prices_db(futbin_id_df)

    time_end = datetime.now()
    time_delta = time_end - time_start
    print("End Time: " + str(time_end))
    print("Time taken to write to DB: " + str(time_delta))
    
    return None

def get_price_stats(f_ytd_avg=0, f_ytd_range=1, f_avg_30d=0, f_range_30d=0,
                    f_avg_14d=1, f_range_14d=1, f_avg_7d=0,f_range_7d=0):
    
    print('Reading hist prices from DB...')
    
    hist_price_db = pd.read_sql(sql='select * from player_prices', con=conn)
    
    print('Calculating hist price stats...')
    
    last_update_date = hist_price_db.groupby('resourceId').last().reset_index()[['resourceId', 'Date']]
    hist_price_db = hist_price_db.pivot(index='resourceId', columns='Date', values='Price')
    hist_price_db.replace(0, np.NaN, inplace=True)
    
    hist_price_db_final = pd.DataFrame()
    
        # create a mix and match list?
    hist_price_db_final['YTD Max'] = hist_price_db.max(axis=1, skipna=True)
    hist_price_db_final['YTD Min'] = hist_price_db.min(axis=1, skipna=True)
    hist_price_db_final['YTD Mean'] = hist_price_db.mean(axis=1, skipna=True)
    hist_price_db_final['YTD Median'] = hist_price_db.median(axis=1, skipna=True)

    hist_price_db_final['14d Max'] = hist_price_db.iloc[:, -14:].max(axis=1, skipna=True)
    hist_price_db_final['14d Min'] = hist_price_db.iloc[:, -14:].min(axis=1, skipna=True)
    hist_price_db_final['14d Mean'] = hist_price_db.iloc[:, -14:].mean(axis=1, skipna=True)
    hist_price_db_final['14d Median'] = hist_price_db.iloc[:, -14:].median(axis=1, skipna=True)

    hist_price_db_final['YTD S.D.'] = hist_price_db.std(axis=1, skipna=True)
    hist_price_db_final['14d S.D.'] = hist_price_db.iloc[:, -14:].std(axis=1, skipna=True)

    hist_price_db_final['Yest'] = hist_price_db[hist_price_db.columns[-1]]
    hist_price_db_final['Last Updated'] = hist_price_db[hist_price_db.columns[-1]]
    
    hist_price_db_final.reset_index(inplace=True)
    
    hist_price_db_final['Name'] = hist_price_db_final['resourceId'].apply(lambda x: fut.core.get_player_name(x))

    hist_price_db_final.set_index(['resourceId', 'Name'], inplace=True)
        # change NAs to 0s
    hist_price_db_final.fillna(0, inplace=True)
    hist_price_db_final = hist_price_db_final.round().astype(int)
    
        # calc C.V. here to avoid conversion to int
    hist_price_db_final['YTD C.V.'] = hist_price_db_final['YTD S.D.'] / hist_price_db_final['YTD Mean']
    hist_price_db_final['14d C.V.'] = hist_price_db_final['14d S.D.'] / hist_price_db_final['14d Mean']
    
    hist_price_db_final['% of 14d Median'] = (hist_price_db_final['Yest'] - hist_price_db_final['14d Mean']) /\
                                                hist_price_db_final['14d Mean']
    hist_price_db_final['14d Z-Score'] = (hist_price_db_final['Yest'] - hist_price_db_final['14d Mean']) /\
                                            hist_price_db_final['14d S.D.']
    
    #hist_price_db_final['% from 14d Median'] = (hist_price_db_final['% from 14d Median'] * 100).astype(int)
    
    hist_price_db_final.reset_index(inplace=True)
    
        # rearrange columns
    hist_price_db_final = hist_price_db_final[['resourceId', 'Name',
                                               'YTD Max', 'YTD Min', 'YTD Mean', 'YTD Median',  
                                               'YTD S.D.', 'YTD C.V.',
                                               '14d Max', '14d Min', '14d Mean', '14d Median',  
                                               '14d S.D.', '14d C.V.', 
                                              'Yest', '% of 14d Median', '14d Z-Score']]
    hist_price_db_final = hist_price_db_final.merge(last_update_date, on='resourceId', how='left') 
    hist_price_db_final.rename(columns={'Date': 'Last Updated'}, inplace=True)
    
    if f_ytd_avg == 0:
        hist_price_db_final.drop(['YTD Mean', 'YTD Median'], axis=1, inplace=True)
    if f_ytd_range == 0:
        hist_price_db_final.drop(['YTD Max','YTD Min'], axis=1, inplace=True)
    
    hist_price_db_final.sort_values('14d Z-Score', inplace=True)
    
    print('Writing price stats to DB...')
    hist_price_db_final.to_sql(name='player_price_stats', index=False, if_exists='replace', con=conn)
    
    # TODO: fix YTD Min = 0 for players; maybe because added late? or not available on market?
    # TODO: write output to DB?
    
    return hist_price_db_final

def futbin_market_scan(min_rating=75, max_rating=99, min_price=200, max_price=15000000):
    ## 
    ## Returns: dataframe with resource_id which fulfils criteria
    ## TODO: enhance to take in list of resource_ids?
    
    num_requests = 0
    
    print('Scanning the market for players rated ' + str(min_rating) + '-' + str(max_rating) + ' between ' + str(min_price)\
         + '-' + str(max_price) + '...')
    
    futbin_id_df = pd.DataFrame(columns=['futbinId', 'Current'])
    
    url = 'https://www.futbin.com/19/players?page={0}&player_rating={1}-{2}&ps_price={3}-{4}&sort=ps_price&order=desc'
    first_page_url = url.format(1, min_rating, max_rating, min_price, max_price)
    
    print(proxies)
    first_page = requests.get(first_page_url, proxies=proxies, timeout=5)
    num_requests += 1
        #sleep
    time.sleep(random.randint(1,3))
    
    first_page_soup = BeautifulSoup(first_page.text, 'lxml')
    
        # get the # of pages to be iterated through
    if first_page_soup.find_all('li', class_='page-item ') == []:
        num_pages = 1
    else:
        num_pages = int(first_page_soup.find_all('li', class_='page-item ')[-1].find('a').find(text=True))
    
    print(str(num_pages) + " pages/requests: " + str(num_pages * 30) + " players expected")
    print('Stick to around 2-3 pages at 1 go')
    
    print('Getting FUTBIN IDs and current prices...')
    
    for i in range(num_pages):
        other_page_url = url.format(i+1, min_rating, max_rating, min_price, max_price) #i+1 to start from page 1 rather than 0
        page = requests.get(other_page_url, proxies=proxies, timeout=5)
        num_requests += 1
            #testing
        print(proxies)
            #sleep
        time.sleep(random.randint(1,3))
        
        soup = BeautifulSoup(page.text, 'lxml')
        table = soup.find('table', class_='table table-bordered table-hover table-responsive w-100 d-block d-md-table')

        for row in table.find_all('tr'):
            player_bio = row.find('td', class_='table-row-text row')
            
                # find url for actual player page
            player_page_url_container = player_bio.find('div', class_='d-inline pt-2 pl-3')
            player_page_url_partial = player_page_url_container.find('a', class_='player_name_players_table').get('href')
            
                ## get futbin_id and map to resource_id in DB
            index = player_page_url_partial.find('/', 11)
            futbin_id = player_page_url_partial[11:index]
            
            #player_page_url = 'https://www.futbin.com' + str(player_page_url_partial).replace(" ", "%20")

            '''current_price = row.find('span', class_='ps4_color font-weight-bold').find(text=True)
            if current_price.find('K') != -1:
                current_price = float(current_price.replace('K', '')) * 1000 #change to int?
            elif current_price.find('M') != -1:
                current_price = float(current_price.replace('M', '')) * 1000000
            else:
                current_price = float(current_price)

            current_price = int(current_price)
    '''
            futbin_id_df = futbin_id_df.append({'futbinId': futbin_id},
                                                  ignore_index=True)
    
    print('No. of requests for this query: ' + str(num_requests))
    
    return futbin_id_df

def get_resource_id(futbin_id):
    
        ## check DB first
    futbin_id_resource_id_db = pd.read_sql(sql='select * from futbin_id_resource_id', con=conn)
    #futbin_id_resource_id_db = pd.DataFrame(columns=['futbinId', 'resourceId', 'Name', 'Rating'])
    
    
    if (futbin_id_resource_id_db['futbinId']==futbin_id).any():
        print('resource_id found in DB; returning resource_id')
        resource_id = int(futbin_id_resource_id_db[futbin_id_resource_id_db['futbinId'] == futbin_id]['resourceId'])
 
    else:
        print('resource_id not found; getting resource_id from FUTBIN...')
        player_page_url = 'https://www.futbin.com/19/player/' + str(futbin_id)
        player_page = requests.get(player_page_url, proxies=proxies, timeout=5)
            #sleep
        time.sleep(random.randint(1,3))

        player_page_soup = BeautifulSoup(player_page.text, 'lxml')
        resource_id_container = player_page_soup.find('div', class_='site-player-page')
        resource_id_container_div = resource_id_container.find('div', class_='container')
        resource_id = int(resource_id_container_div.find('div').get('data-player-resource'))


        futbin_id_resource_id_db = futbin_id_resource_id_db.append({'futbinId': futbin_id,
                                                             'resourceId': resource_id,
                                                            'Name': fut.core.get_player_name(resource_id),
                                                            'Rating': rating}, ignore_index=True)

        futbin_id_resource_id_db.drop_duplicates(inplace=True)

        print('Appending futbin_id & resource_id to DB...', end = " ")
        futbin_id_resource_id_db.to_sql(name='futbin_id_resource_id', index=False, if_exists='replace', con=conn)
        print('Success!')
        print('-----------------------------------------')
     
    #display(futbin_id_resource_id_db)

    return resource_id


# this method is causing issues, only keep last day's price data
def drop_duplicates_db():
    
    hist_price_db = pd.read_sql(sql='select * from player_prices', con=conn)
    hist_price_db.drop_duplicates(keep='last', inplace=True) 
    hist_price_db.to_sql(name='player_prices', index=False, if_exists='replace', con=conn)
    
    return None

def write_futbin_hist_prices_db(futbin_id_df):
    print('Loading existing hist prices from DB...')
    
    hist_price_db = pd.read_sql(sql='select * from player_prices', con=conn)
    hist_price_stats = get_price_stats(hist_price_db)
        
    hist_price_df = pd.DataFrame(columns=['resourceId', 'Date', 'Price'])
    
    futbin_id_df['futbinId'] = futbin_id_df['futbinId'].astype(int) # change to 'int' so that baseId can take in
    
    yesterday = (datetime.utcnow() - timedelta(1)).strftime('%Y-%m-%d')
       
    
        # want to check if df ids are in db
    for index, row in futbin_id_df.iterrows():
            # resource_id
        resource_id = get_resource_id(row['futbinId']) ## function 
        name = fut.core.get_player_name(resource_id)
        
        if resource_id in hist_price_db['resourceId'].unique() and \
            yesterday in hist_price_db[hist_price_db['resourceId']==resource_id]['Date'].unique():
            print(name + " not added; yest price data already exists in DB")
            print('-----------------------------------------')
            pass
        else:
                #hist price data 
            hist_price_url = 'https://www.futbin.com/19/playerGraph?type=daily_graph&year=19&player={0}'
            hist_price_data = requests.get(hist_price_url.format(resource_id), proxies=proxies, timeout=5).json()

                # sleep for 5-10 seconds
            time.sleep(random.randint(3,5))

            for price in hist_price_data['ps']:
                #There is extra zeroes in response.
                date = datetime.utcfromtimestamp(price[0] / 1000).strftime('%Y-%m-%d')
                price = price[1]

                hist_price_df = hist_price_df.append({'resourceId': resource_id,
                                                        'Date': date, 
                                                        'Price': price}, ignore_index=True)
            
                ## shift this to write to DB per player so that won't lose data if interrupted
            hist_price_df.to_sql(name='player_prices', index=False, if_exists='append', con=conn)
            print(name + " added to DB")
            print('-----------------------------------------')
            
    print('Dropping duplicates...')
    #drop_duplicates_db()
    print('-----------------------------------------')
    
    return None


def display_id_map():
    df = pd.read_sql(sql='select * from futbin_id_resource_id', con=conn)
    return df

def get_hourly_prices():
    
    return None

In [99]:
drop_duplicates_db()

In [108]:
display_id_map()

,futbinId,resourceId
0,17575,100684097
1,17576,184707399
2,167,181783
3,395,192638
4,447,209658
5,479,50526734
6,811,50511495
7,1164,50541683
8,1204,50524241
9,1248,50543396


In [35]:
futbin_id_df = futbin_market_scan(min_rating=82, max_rating=82, min_price=1000, max_price=3000)
futbin_id_df

Scanning the market for players rated 82-82 between 1000-3000...
{'http': 'http://68.183.97.187:3128'}
3 pages/requests: 90 players expected
Stick to around 2-3 pages at 1 go
Getting FUTBIN IDs...
{'http': 'http://68.183.97.187:3128'}
{'http': 'http://68.183.97.187:3128'}
{'http': 'http://68.183.97.187:3128'}
No. of requests for this query: 4


,futbinId
0,259
1,732
2,870
3,936
4,1334
5,2042
6,869
7,1335
8,1685
9,368


In [33]:
get_price_stats()

Reading hist prices from DB...
Calculating hist price stats...
Writing price stats to DB...


C:\Anaconda3\lib\site-packages\pandas\core\generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


,resourceId,Name,YTD Max,YTD Min,YTD S.D.,YTD C.V.,14d Max,14d Min,14d Mean,14d Median,14d S.D.,14d C.V.,Yest,% of 14d Median,14d Z-Score,Last Updated
0,5479,Casillas,1867,1867,0,0.00,1867,1867,1867,1867,0,0.00,0,-1.00,-inf,2019-05-05
189,50471941,Antonio Mirante,987,987,0,0.00,987,987,987,987,0,0.00,0,-1.00,-inf,2019-05-06
190,50478408,Juanfran,2677,2677,0,0.00,2677,2677,2677,2677,0,0.00,0,-1.00,-inf,2019-05-06
191,50479767,Igor Akinfeev,1182,1182,0,0.00,1182,1182,1182,1182,0,0.00,0,-1.00,-inf,2019-05-06
192,50488167,Héctor Herrera,2104,2104,0,0.00,2104,2104,2104,2104,0,0.00,0,-1.00,-inf,2019-05-05
193,50489608,Manuel Fernandes,2206,2206,0,0.00,2206,2206,2206,2206,0,0.00,0,-1.00,-inf,2019-05-05
194,50489781,Jefferson Farfán,1020,1020,0,0.00,1020,1020,1020,1020,0,0.00,0,-1.00,-inf,2019-05-05
195,50493604,Borja Valero,2058,2058,0,0.00,2058,2058,2058,2058,0,0.00,0,-1.00,-inf,2019-05-05
196,50502016,Erik Lamela,2797,2797,0,0.00,2797,2797,2797,2797,0,0.00,0,-1.00,-inf,2019-05-06
197,50503481,Daniel Sturridge,2539,2539,0,0.00,2539,2539,2539,2539,0,0.00,0,-1.00,-inf,2019-05-06


In [109]:
url1 = 'https://www.easports.com/fifa/ultimate-team/api/fut/item?ovr=99:99'
data = requests.get(url1, proxies=proxies, timeout=5).json()
num_pages = data['totalPages']
#num_results_page =  data['count']
num_results_total = data['totalResults']
data



{'count': 4,
 'items': [{'acceleration': 95,
   'age': 34,
   'aggression': 75,
   'agility': 94,
   'atkWorkRate': 'High',
   'attributes': [{'chemistryBonus': [0],
     'name': 'fut.attribute.PAC',
     'value': 97},
    {'chemistryBonus': [0], 'name': 'fut.attribute.SHO', 'value': 99},
    {'chemistryBonus': [0], 'name': 'fut.attribute.PAS', 'value': 95},
    {'chemistryBonus': [0], 'name': 'fut.attribute.DRI', 'value': 98},
    {'chemistryBonus': [0], 'name': 'fut.attribute.DEF', 'value': 50},
    {'chemistryBonus': [0], 'name': 'fut.attribute.PHY', 'value': 95}],
   'balance': 76,
   'ballcontrol': 99,
   'baseId': 20801,
   'birthdate': '02/05/1985',
   'club': {'abbrName': 'Juventus',
    'id': 45,
    'imageUrls': {'dark': {'large': 'https://www.easports.com/fifa/ultimate-team/web-app/content/7D49A6B1-760B-4491-B10C-167FBC81D58A/2019/fut/items/images/mobile/clubs/dark/45.png',
      'medium': 'https://www.easports.com/fifa/ultimate-team/web-app/content/7D49A6B1-760B-4491-B10C-1

In [11]:
list(data.keys())

['page', 'totalPages', 'totalResults', 'type', 'count', 'items']

In [34]:
data['items'][0].keys()

dict_keys(['commonName', 'firstName', 'lastName', 'league', 'nation', 'club', 'headshot', 'position', 'composure', 'playStyle', 'playStyleId', 'height', 'weight', 'birthdate', 'age', 'acceleration', 'aggression', 'agility', 'balance', 'ballcontrol', 'foot', 'skillMoves', 'crossing', 'curve', 'dribbling', 'finishing', 'freekickaccuracy', 'gkdiving', 'gkhandling', 'gkkicking', 'gkpositioning', 'gkreflexes', 'headingaccuracy', 'interceptions', 'jumping', 'longpassing', 'longshots', 'marking', 'penalties', 'positioning', 'potential', 'reactions', 'shortpassing', 'shotpower', 'slidingtackle', 'sprintspeed', 'standingtackle', 'stamina', 'strength', 'vision', 'volleys', 'weakFoot', 'traits', 'specialities', 'atkWorkRate', 'defWorkRate', 'playerType', 'attributes', 'name', 'rarityId', 'isIcon', 'quality', 'isGK', 'positionFull', 'isSpecialType', 'contracts', 'fitness', 'rawAttributeChemistryBonus', 'isLoan', 'squadPosition', 'iconAttributes', 'itemType', 'discardValue', 'id', 'modelName', 'bas

In [133]:
import ast
attributes_list = ast.literal_eval(data['items'][0]['attributes'])
attributes_list

[{'chemistryBonus': [0], 'name': 'fut.attribute.PAC', 'value': 97},
 {'chemistryBonus': [0], 'name': 'fut.attribute.SHO', 'value': 99},
 {'chemistryBonus': [0], 'name': 'fut.attribute.PAS', 'value': 95},
 {'chemistryBonus': [0], 'name': 'fut.attribute.DRI', 'value': 98},
 {'chemistryBonus': [0], 'name': 'fut.attribute.DEF', 'value': 50},
 {'chemistryBonus': [0], 'name': 'fut.attribute.PHY', 'value': 95}]

In [218]:
df = pd.DataFrame()

for attributes in attributes_list:
    df = df.append(attributes, ignore_index=True)
    
df.drop('chemistryBonus', axis=1, inplace=True)
df['value'] = df['value'].astype(int)
df['name'] = df['name'].apply(lambda x: x[-3:])

In [223]:
df

,name,value
0,PAC,97
1,SHO,99
2,PAS,95
3,DRI,98
4,DEF,50
5,PHY,95


In [225]:
df.pivot(columns='name', values='value')

name,DEF,DRI,PAC,PAS,PHY,SHO
0,nan,nan,97.00,nan,nan,nan
1,nan,nan,nan,nan,nan,99.00
2,nan,nan,nan,95.00,nan,nan
3,nan,98.00,nan,nan,nan,nan
4,50.00,nan,nan,nan,nan,nan
5,nan,nan,nan,nan,95.00,nan


In [215]:
df.pivot()#.rename(columns=df.iloc[1])#.drop(df.index[0])

,0,1,2,3,4,5
name,PAC,SHO,PAS,DRI,DEF,PHY
value,97,99,95,98,50,95


In [207]:
df

,name,value
0,PAC,97
1,SHO,99
2,PAS,95
3,DRI,98
4,DEF,50
5,PHY,95


In [125]:
columns = data['items'][0].keys()

#columns=list(columns)
df = pd.DataFrame()

for col in list(columns):
    #df[col] = data['items'][0][col]
    
    if col == 'attributes':
        print(type(col))
        #for dict in data['items'][0][col]:
            #print(dict)
            #print(attribute)
            #print(attribute.get('name')[-3:] + ': ' + attribute.get('value'))
    
    #elif isinstance(columns, dict):
       # pass
   # elif isinstance(columns, list):
   #     for things in columns:
   #         print(things + ": " + data['items'][0][col][things])
   # else:
        #data['items'][0][col] = str(data['items'][0][col])
        #print(col + ': '  + str(data['items'][0][col]))
        


<class 'str'>


In [20]:
data['items']

[{'acceleration': 68,
  'age': 35,
  'aggression': 72,
  'agility': 52,
  'atkWorkRate': 'High',
  'attributes': [{'chemistryBonus': [0],
    'name': 'fut.attribute.PAC',
    'value': 71},
   {'chemistryBonus': [0], 'name': 'fut.attribute.SHO', 'value': 64},
   {'chemistryBonus': [0], 'name': 'fut.attribute.PAS', 'value': 72},
   {'chemistryBonus': [0], 'name': 'fut.attribute.DRI', 'value': 63},
   {'chemistryBonus': [0], 'name': 'fut.attribute.DEF', 'value': 84},
   {'chemistryBonus': [0], 'name': 'fut.attribute.PHY', 'value': 82}],
  'balance': 43,
  'ballcontrol': 62,
  'baseId': 112316,
  'birthdate': '10/29/1983',
  'club': {'abbrName': 'Sporting',
   'id': 237,
   'imageUrls': {'dark': {'large': 'https://www.easports.com/fifa/ultimate-team/web-app/content/7D49A6B1-760B-4491-B10C-167FBC81D58A/2019/fut/items/images/mobile/clubs/dark/237.png',
     'medium': 'https://www.easports.com/fifa/ultimate-team/web-app/content/7D49A6B1-760B-4491-B10C-167FBC81D58A/2019/fut/items/images/mobile

In [5]:
for i in range(num_pages-1):
    print(i)
    url2 = 'https://www.easports.com/fifa/ultimate-team/api/fut/item?ovr=81:81&page={i+1}'
    data2 = requests.get(url2, proxies=proxies, timeout=5).json()
    
    for player in data2: 
        #don't bother creating so many variables, just append directly i.e. df['Name'] = player['name] or sth
        name = player['name']
        attribute_list = player['attributes']

       #attributes list
        for index in range(len(attribute_list)): #this returns 0-5
            for attribute in attribute_list[index]:
                if index == 0:
                    pace = attribute_list[index]['value']
                elif index == 1:
                    shooting = attribute_list[index]['value']
                elif index == 2:
                    passing = attribute_list[index]['value']
                elif index == 3:
                    dribbling = attribute_list[index]['value']
                elif index == 4:
                    defending = attribute_list[index]['value']
                elif index == 5:
                    physical = attribute_list[index]['value']


        id = player['id'] #100684097 for 99 c.ronaldo; this id for back end json prices; resource_id?
        asset_id = player['baseId'] #20801 for 99 c.ronaldo; asset_id?
        #futbin uses 17575 for 99 c. ronaldo front end

        rating = player['rating']
        position = player['position']
        version = player['rarityId']

{'count': 24,
 'items': [{'acceleration': 68,
   'age': 35,
   'aggression': 72,
   'agility': 52,
   'atkWorkRate': 'High',
   'attributes': [{'chemistryBonus': [0],
     'name': 'fut.attribute.PAC',
     'value': 71},
    {'chemistryBonus': [0], 'name': 'fut.attribute.SHO', 'value': 64},
    {'chemistryBonus': [0], 'name': 'fut.attribute.PAS', 'value': 72},
    {'chemistryBonus': [0], 'name': 'fut.attribute.DRI', 'value': 63},
    {'chemistryBonus': [0], 'name': 'fut.attribute.DEF', 'value': 84},
    {'chemistryBonus': [0], 'name': 'fut.attribute.PHY', 'value': 82}],
   'balance': 43,
   'ballcontrol': 62,
   'baseId': 112316,
   'birthdate': '10/29/1983',
   'club': {'abbrName': 'Sporting',
    'id': 237,
    'imageUrls': {'dark': {'large': 'https://www.easports.com/fifa/ultimate-team/web-app/content/7D49A6B1-760B-4491-B10C-167FBC81D58A/2019/fut/items/images/mobile/clubs/dark/237.png',
      'medium': 'https://www.easports.com/fifa/ultimate-team/web-app/content/7D49A6B1-760B-4491-B1

In [ ]:
# multi-processing